In [6]:
CARTA_CDNA = """
  <main class="hoja">
    <header class="doc-header">
      <div class="header-top">
        <img class="logo" src="images/LOGO_home.jpg" alt="Logo">
      </div>

          <div class="fecha">
        Santiago de Surco, {{FECHA_HOY}}
      </div>
    </header>

    <div class="titulo">CONSTANCIA DE NO ADEUDO - {{DNI}}</div>

    <section class="saludo">
      <p>A QUIEN CORRESPONDA</p>
    </section>

    <section class="cuerpo">
      <p>
        Por medio de la presente, le informamos que EXPÉRTIS MASTER SERVICER AND COLLECTIONS S.A.C., identificada con R.U.C. Nº 20600558481, con domicilio en Av. Circunvalación del Club Golf Los Incas N°154 - Oficina 403, distrito de Santiago de Surco, provincia y departamento de Lima, debidamente representada por Nathaly Isabel Jimenez Quispe según poderes inscritos en la partida N° 13459121, ha adquirido los derechos de la cartera de crédito otorgado por {{ENTIDAD}}, con base en el contrato de transferencia de cartera suscrito el {{FECHA_CASTIGO}}.
      </p>
      <p>
        Al respecto informamos que ├├el┤┤╞╞los╡╡ ├├crédito┤┤╞╞créditos╡╡ {{CREDITOS}} ├├correspondiente┤┤╞╞correspondientes╡╡ al Sr.(a) {{NOMBRE}}, identificado(a) con DNI N° {{DNI}}, ├├ha┤┤ ╞╞han╡╡ sido ├├cancelado┤┤ ╞╞cancelados╡╡ en su totalidad el día {{FECHA_PAGO}}, no existiendo a la fecha saldo pendiente de pago, por lo que se da por cancelada la deuda. Cabe mencionar que si ├├el┤┤ ╞╞los╡╡ ├├crédito┤┤ ╞╞créditos╡╡ en mención ├├cuenta┤┤ ╞╞cuentan╡╡ con un proceso judicial será responsabilidad única y exclusivamente del cliente asumir y realizar de manera total con todas las gestiones y los gastos notariales, registrales, tributarios y otros pertinentes que originen la formalización de la conclusión del proceso judicial, levantamiento de embargo y/o levantamiento de hipoteca¹.
      </p>
      <p>
        Se expide en Lima el {{FECHA_HOY}}, y se imprimió en Lima el {{FECHA_HOY}}.
      </p>
      <p>
        Sin otro particular, quedamos de ustedes.
      </p>
      <p>Atentamente,</p>
    </section>

    <section class="despedida">
      <p>EXPÉRTIS MASTER SERVICER & COLLECTIONS SAC</p>
    </section>

    <!-- FIRMA (imagen) -->
    <section class="firma">
      <img class="firma-img" src="images/FIRMA_nathaly.jpg" alt="Firma">
      <div class="firma-nombre">Nathaly Isabel Jimenez Quispe</div>
      <div class="firma-cargo">Apoderada</div>
    </section>

    <section class="nota">
      <small>
        (1) Para concluir cualquier proceso judicial deberá interponerse el trámite correspondiente, que ayudará al cliente a reducir el trámite de manera idónea. Por el lado de la hipoteca se tendrá en cuenta que ha sido judicial, además, se solicita la cuenta de levantamiento del gravamen.
      </small>
    </section>

  </main>
"""

In [11]:
from pathlib import Path
import re, json

ruta_index = Path(r"C:\Users\Jorge Vasquez\DocoBr\index.html")

# 1) Detectar placeholders automáticamente desde CARTA_CDNA
placeholders = sorted(set(re.findall(r"\{\{\s*([A-Za-z0-9_]+)\s*\}\}", CARTA_CDNA)))
placeholders_json = json.dumps(placeholders, ensure_ascii=False)

# 2) Labels / defaults (opcionales)
LABELS = {
    "FECHA_HOY": "Fecha de emisión",
    "DNI": "DNI",
    "NOMBRE": "Nombre completo",
    "ENTIDAD": "Entidad (banco / empresa)",
    "FECHA_CASTIGO": "Fecha de cesión / castigo",
    "CREDITOS": "Créditos (lista o códigos)",
    "FECHA_PAGO": "Fecha de pago",
}

TEXTAREA_KEYS = set()
DEFAULTS = {}

SELECT_KEYS = {"ENTIDAD"}

ENTIDAD_OPTIONS = [
    "Caja Municipal de Ahorro y Crédito de Arequipa S.A.",
    "Banco de la Nación",
    "Banco BBVA Perú",
    "Financiera Credinka S.A.",
    "Caja Rural de Ahorro y Crédito del Centro S.A.",
    "Banco Internacional del Perú S.A.A. – Interbank",
    "Caja Rural de Ahorro y Crédito Los Andes S.A.",
    "Mibanco – Banco de la Microempresa S.A.",
    "Financiera ProEmpresa S.A.",
    "Financiera Qaqap S.A.C.",
    "Caja Municipal de Ahorro y Crédito de Sullana S.A.",
    "Banco Scotiabank Perú S.A.A.",
]

def make_field(key: str) -> str:
    label = LABELS.get(key, key)
    default = DEFAULTS.get(key, "")

    if key == "CREDITOS":
        return f"""
        <div class="field field-creditos">
          <div class="creditos-header">
            <label>Créditos (lista o códigos)</label>

            <div class="creditos-selector">
              <span>Cantidad</span>
              <select id="CREDITOS_COUNT">
                <option value="1" selected>1</option>
                <option value="2">2</option>
                <option value="3">3</option>
                <option value="4">4</option>
                <option value="5">5</option>
              </select>
            </div>
          </div>

          <div class="creditos-grid" id="CREDITOS_CONTAINER">
            <input id="CREDITOS_1" type="text" placeholder="Crédito 1">
            <input id="CREDITOS_2" type="text" placeholder="Crédito 2">
            <input id="CREDITOS_3" type="text" placeholder="Crédito 3">
            <input id="CREDITOS_4" type="text" placeholder="Crédito 4">
            <input id="CREDITOS_5" type="text" placeholder="Crédito 5">
          </div>
        </div>
        """.strip()

    if key in SELECT_KEYS:
        opts = ['<option value="">Selecciona una entidad...</option>']
        for opt in ENTIDAD_OPTIONS:
            selected = ' selected' if default and default == opt else ''
            opts.append(f'<option value="{opt}"{selected}>{opt}</option>')
        options_html = "\n".join(opts)

        return f"""
        <div class="field">
          <label for="{key}">{label}</label>
          <select id="{key}">
            {options_html}
          </select>
        </div>
        """.strip()

    if key in TEXTAREA_KEYS:
        return f"""
        <div class="field">
          <label for="{key}">{label}</label>
          <textarea id="{key}" rows="3" placeholder="Ingresa {label}">{default}</textarea>
        </div>
        """.strip()

    return f"""
    <div class="field">
      <label for="{key}">{label}</label>
      <input id="{key}" type="text" value="{default}" placeholder="Ingresa {label}">
    </div>
    """.strip()

fields_html = "\n".join(make_field(k) for k in placeholders)

# Evitar cierre accidental de template
carta_cdna_safe = CARTA_CDNA.replace("</template>", "<\\/template>")

CSS_BASE = """
<style>
    /* =======================
       CONFIGURACIÓN GLOBAL
    ======================= */
    @page{
      size: Letter;   /* 21.59 cm x 27.94 cm */
      margin: 0;
    }

    body{
      margin:0;
      background:#f5f5f5;
      font-family: Arial, Helvetica, sans-serif;
      line-height:1.6;
      color:#222;
    }

    /* =======================
       HOJA LETTER (DOCUMENTO)
    ======================= */
    .hoja{
      width:21.59cm;
      height:27.94cm;

      margin:40px auto;
      background:#fff;

      /* márgenes internos del documento */
      padding:2.5cm 2cm;

      box-sizing:border-box;
      position:relative;

      /* evita crecimiento por contenido */
      overflow:hidden;

      /* elimina estilos web */
      border-radius:0;
      box-shadow:none;
    }

    /* =======================
       ENCABEZADO CON LOGO
    ======================= */
    .doc-header{
      margin-bottom: 6px;
    }

    .header-top{
      display:flex;
      justify-content:flex-end;
      align-items:flex-start;
      margin-bottom: 6px;
    }

    .logo{
      max-width: 150px;   /* ajusta si lo quieres más grande/pequeño */
      height: auto;
      display:block;
    }

    /* =======================
       CONTENIDO DE CARTA
    ======================= */
    .fecha{
      text-align:right;
      white-space:nowrap;
      font-size:11px;
    }

    .titulo{
      text-align:center;
      margin:18px 0 26px;
      padding:10px 14px;
      font-weight:700;
      font-size:20px;
      text-decoration:underline;
    }

    .saludo{
      margin:10px 0 18px;
      font-weight:700;
      font-size:11.5px;
    }

    .cuerpo{
      font-size:11.5px;
    }

    .cuerpo p{
      margin:0 0 14px;
      text-align:justify;
    }

    .despedida{
      margin-top:24px;
      font-size:11.5px;
    }

    /* =======================
       FIRMA (IMAGEN)
    ======================= */
    .firma{
      margin-top: 46px;
      text-align:center;
    }

    .firma-img{
      max-width: 230px;  /* ajusta tamaño firma */
      height:auto;
      display:block;
      margin: 0 auto 6px;
    }

    .firma-nombre{
      font-weight:700;
      font-size:11.5px;
      margin-top:2px;
    }

    .firma-cargo{
      font-size:11px;
      margin-top:2px;
    }

    .nota{
      margin-top: 18px;
    }

    .nota small{
      display:block;
      color:#666;
      font-size:10px;
      line-height:1.35;
      text-align:justify;
    }

    /* =======================
       UI FORMULARIO (SIN CAMBIOS)
    ======================= */
    .app-bar{
      max-width:780px;
      margin:40px auto 0;
      padding:0 12px;
      display:flex;
      justify-content:space-between;
      align-items:center;
      gap:12px;
    }

    .brand{
      font-weight:800;
      letter-spacing:.2px;
      color:#111;
    }

    .panel{
      max-width:780px;
      margin:12px auto 0;
      background:#fff;
      border-radius:10px;
      box-shadow:0 8px 20px rgba(0,0,0,.08);
      padding:20px 18px;
    }

    .grid{
      display:grid;
      grid-template-columns: 1fr 1fr;
      gap:12px;
    }

    @media (max-width: 760px){
      .grid{ grid-template-columns: 1fr; }
    }

    /* =======================
       FORM CONTROLS
    ======================= */
    .field label{
      display:block;
      font-size:12px;
      color:#444;
      margin-bottom:6px;
      font-weight:700;
    }

    .field input,
    .field textarea,
    .field select{
      width:100%;
      box-sizing:border-box;
      border:1px solid #ddd;
      border-radius:8px;
      padding:7px 12px;
      font-size:14px;
      outline:none;
    }

    .mini-label{
      display:block;
      font-size:12px;
      color:#444;
      margin-bottom:6px;
      font-weight:700;
    }

    /* =======================
       CRÉDITOS
    ======================= */
    .creditos-toolbar{
      display:flex;
      gap:12px;
      align-items:flex-end;
      margin-bottom:10px;
    }

    .creditos-grid{
      display:grid;
      grid-template-columns: 1fr 1fr;
      gap:10px;
    }

    @media (max-width: 760px){
      .creditos-grid{ grid-template-columns: 1fr; }
    }

    .creditos-header{
      display:flex;
      justify-content:space-between;
      align-items:center;
      margin-bottom:10px;
    }

    .creditos-header > label{
      font-size:12px;
      font-weight:700;
      color:#444;
    }

    .creditos-selector{
      display:flex;
      align-items:center;
      gap:6px;
      font-size:12px;
      color:#555;
    }

    .creditos-selector select{
      width:auto;
      padding:4px 8px;
      font-size:12px;
      border-radius:6px;
    }

    /* =======================
       ACCIONES
    ======================= */
    .actions{
      display:flex;
      gap:10px;
      justify-content:flex-end;
      margin-top:14px;
    }

    button{
      border:0;
      border-radius:10px;
      padding:10px 14px;
      font-weight:800;
      cursor:pointer;
    }

    .btn-primary{
      background:#111;
      color:#fff;
    }

    .btn-ghost{
      background:#eee;
      color:#111;
    }

    .hint{
      font-size:12px;
      color:#666;
      margin:10px 0 0;
    }

    .hidden{
      display:none !important;
    }

    @media print {
      body { background: #fff !important; }
      .app-bar, #panelFormulario { display: none !important; }
      #panelCarta { display: block !important; }
      .hoja {
        margin: 0 !important;
        box-shadow: none !important;
        width: 21.59cm !important;
        height: 27.94cm !important;
      }
    }
    
    @page {
      size: Letter;
      margin: 0;
    }
</style>
""".strip()

HTML_BASE = """
<!doctype html>
<html lang="es">
<head>
  <meta charset="utf-8" />
  <meta name="viewport" content="width=device-width, initial-scale=1" />
  <title>DOCUMENTOS FAST</title>
  %%CSS%%
  <script src="https://cdn.jsdelivr.net/npm/html2canvas@1.4.1/dist/html2canvas.min.js"></script>
  <script src="https://cdn.jsdelivr.net/npm/jspdf@2.5.1/dist/jspdf.umd.min.js"></script>
</head>
<body>

  <div class="app-bar">
      <div class="brand">DOCUMENTOS FAST</div>
      <div>
        <button class="btn-ghost" id="btnEditar" type="button">Editar datos</button>
        <button class="btn-primary hidden" id="btnImprimir" type="button">GUARDAR PDF</button>
      </div>
  </div>

  <!-- Formulario -->
  <section class="panel" id="panelFormulario">
    <form id="formCarta">
      <div class="grid">
        %%FIELDS%%
      </div>

      <div class="actions">
        <button class="btn-primary" type="submit">GENERAR CARTA</button>
      </div>

      <p class="hint">Completa los campos y presiona <b>GENERAR CARTA</b> para ver la carta.</p>
    </form>
  </section>

  <!-- Carta -->
  <section id="panelCarta" class="hidden">
    <div id="renderCarta"></div>
  </section>

  <template id="tplCarta">
%%CARTA_CDNA%%
  </template>

<script>
  const placeholders = %%PLACEHOLDERS%%;
  const creditosCount = document.getElementById('CREDITOS_COUNT');
  const creditosContainer = document.getElementById('CREDITOS_CONTAINER');
  const panelFormulario = document.getElementById('panelFormulario');
  const panelCarta = document.getElementById('panelCarta');
  const renderCarta = document.getElementById('renderCarta');
  const tplCarta = document.getElementById('tplCarta');
  const btnEditar = document.getElementById('btnEditar');
  const btnDescargar = document.getElementById('btnDescargar'); // ✅ aquí
  const form = document.getElementById('formCarta');

  function showForm() {
    panelFormulario.classList.remove('hidden');
    panelCarta.classList.add('hidden');
    btnDescargar?.classList.add('hidden'); // ✅ seguro
    window.scrollTo({ top: 0, behavior: 'smooth' });
  }

  function showCarta() {
    panelFormulario.classList.add('hidden');
    panelCarta.classList.remove('hidden');
    window.scrollTo({ top: 0, behavior: 'smooth' });
  }

  function syncCreditosInputs() {
    if (!creditosCount || !creditosContainer) return;
    const n = Math.max(1, Math.min(5, Number(creditosCount.value || 1)));
    for (let i = 1; i <= 5; i++) {
      const inp = document.getElementById(`CREDITOS_${i}`);
      if (!inp) continue;
      const visible = i <= n;
      inp.classList.toggle('hidden', !visible);
      if (!visible) inp.value = '';
    }
  }

  function buildCreditosString() {
    if (!creditosCount) return '';
    const n = Math.max(1, Math.min(5, Number(creditosCount.value || 1)));
    const vals = [];
    for (let i = 1; i <= n; i++) {
      const v = (document.getElementById(`CREDITOS_${i}`)?.value || '').trim();
      if (v) vals.push(v);
    }
    return vals.join(', ');
  }

  function applySingularPluralByCreditos(text) {
    const n = Number(document.getElementById('CREDITOS_COUNT')?.value || 1);

    if (n === 1) {
      text = text.replace(/╞╞[\s\S]*?╡╡/g, '');
      text = text.replace(/├├/g, '').replace(/┤┤/g, '');
    } else {
      text = text.replace(/├├[\s\S]*?┤┤/g, '');
      text = text.replace(/╞╞/g, '').replace(/╡╡/g, '');
    }

    return text;
  }

  function applyTemplate(templateStr, data) {
    return templateStr.replace(/\{\{\s*([A-Za-z0-9_]+)\s*\}\}/g, (match, key) => {
      const v = data[key];
      return (v === undefined || v === null || v === '') ? match : String(v);
    });
  }

  function collectData() {
    const data = {};
    for (const key of placeholders) {
      if (key === 'CREDITOS') {
        data[key] = buildCreditosString();
        continue;
      }
      const el = document.getElementById(key);
      if (!el) continue;
      data[key] = (el.value || '').trim();
    }
    return data;
  }

  btnEditar?.addEventListener('click', showForm);
  creditosCount?.addEventListener('change', syncCreditosInputs);
  syncCreditosInputs();

  form.addEventListener('submit', (e) => {
    e.preventDefault();
    const data = collectData();

    if (('FECHA_HOY' in data) && !data.FECHA_HOY) {
      const d = new Date();
      const dd = String(d.getDate()).padStart(2, '0');
      const mm = String(d.getMonth() + 1).padStart(2, '0');
      const yyyy = d.getFullYear();
      data.FECHA_HOY = `${dd}/${mm}/${yyyy}`;
    }

    const tplRaw = tplCarta.innerHTML;
    const tplFixed = applySingularPluralByCreditos(tplRaw);
    renderCarta.innerHTML = applyTemplate(tplFixed, data);

    showCarta();
    btnDescargar?.classList.remove('hidden'); // ✅ mostrar botón
  });

  btnDescargar?.addEventListener('click', async () => {
    const hoja = document.querySelector('.hoja');
    if (!hoja) return;

    btnDescargar.classList.add('hidden');

    const canvas = await html2canvas(hoja, {
      scale: 2,
      useCORS: true,
      backgroundColor: '#ffffff'
    });

    const imgData = canvas.toDataURL('image/jpeg', 0.95);

    const { jsPDF } = window.jspdf;
    const pdf = new jsPDF({
      orientation: 'portrait',
      unit: 'pt',
      format: 'letter'
    });

    const pageWidth = pdf.internal.pageSize.getWidth();
    const pageHeight = pdf.internal.pageSize.getHeight();

    const imgWidth = pageWidth;
    const imgHeight = (canvas.height * imgWidth) / canvas.width;

    const y = Math.max(0, (pageHeight - imgHeight) / 2);
    pdf.addImage(imgData, 'JPEG', 0, y, imgWidth, imgHeight);

    const dni = (document.getElementById('DNI')?.value || 'SIN_DNI').trim();
    pdf.save(`Constancia_No_Adeudo_${dni}.pdf`);

    btnDescargar.classList.remove('hidden');
  });
</script>

</body>
</html>
""".strip()

html_final = (
    HTML_BASE
    .replace("%%CSS%%", CSS_BASE)
    .replace("%%FIELDS%%", fields_html)
    .replace("%%CARTA_CDNA%%", carta_cdna_safe.strip("\n"))
    .replace("%%PLACEHOLDERS%%", placeholders_json)
)

ruta_index.write_text(html_final, encoding="utf-8")
print(f"✅ index.html actualizado correctamente en:\n{ruta_index}")

✅ index.html actualizado correctamente en:
C:\Users\Jorge Vasquez\DocoBr\index.html


<>:404: SyntaxWarning: invalid escape sequence '\s'
<>:404: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Jorge Vasquez\AppData\Local\Temp\ipykernel_13912\77470340.py:404: SyntaxWarning: invalid escape sequence '\s'
  HTML_BASE = """


In [12]:
import subprocess, os
from pathlib import Path
from datetime import datetime

# === CONFIG (según lo que me diste) ===
REPO_DIR = Path(r"C:\Users\Jorge Vasquez\DocoBr")
REPO_URL = "https://github.com/jvasquez98/DocoBr.git"  # ✅ con .git
BRANCH = "main"
COMMIT_MSG = ""  # si está vacío, se genera automáticamente
FILES_TO_COMMIT = ["index.html", "GenerateUI.ipynb", "images"]  # solo estos

def run(cmd, check=True):
    print(">", " ".join(cmd))
    return subprocess.run(cmd, cwd=str(REPO_DIR), check=check, text=True)

# Validaciones básicas
if not REPO_DIR.exists():
    raise FileNotFoundError(f"No existe la ruta del proyecto: {REPO_DIR}")

for item in FILES_TO_COMMIT:
    p = REPO_DIR / item
    if not p.exists():
        raise FileNotFoundError(f"No existe para subir: {p}")

# 1) Init git si no existe
git_dir = REPO_DIR / ".git"
if not git_dir.exists():
    run(["git", "init"])

# 2) Configurar remote origin (crear o actualizar)
#    (si ya existía con otra URL, lo corrige)
remotes = subprocess.run(["git", "remote"], cwd=str(REPO_DIR), text=True, capture_output=True)
remote_list = (remotes.stdout or "").split()

if "origin" not in remote_list:
    run(["git", "remote", "add", "origin", REPO_URL])
else:
    run(["git", "remote", "set-url", "origin", REPO_URL])

# 3) Cambiar a rama main (crear si no existe)
#    - Si estás en otra rama, te mueve.
#    - Si main no existe, la crea.
branches = subprocess.run(["git", "branch", "--list", BRANCH], cwd=str(REPO_DIR), text=True, capture_output=True)
if (branches.stdout or "").strip() == "":
    run(["git", "checkout", "-b", BRANCH])
else:
    run(["git", "checkout", BRANCH])

# 4) Add SOLO lo que quieres subir
#    (Opcionalmente puedes remover otros cambios del stage si existían)
run(["git", "reset"])
run(["git", "add"] + FILES_TO_COMMIT)

# 5) Commit (si hay cambios)
status = subprocess.run(["git", "status", "--porcelain"], cwd=str(REPO_DIR), text=True, capture_output=True)
if not (status.stdout or "").strip():
    print("✅ No hay cambios para commitear. (Todo ya está actualizado)")
else:
    if not COMMIT_MSG.strip():
        COMMIT_MSG = f"chore: update carta UI + pdf download ({datetime.now().strftime('%Y-%m-%d %H:%M')})"
    run(["git", "commit", "-m", COMMIT_MSG])

# 6) Push
#    - set upstream por si es la primera vez en main
try:
    run(["git", "push", "-u", "origin", BRANCH])
    print("✅ Push completado.")
except subprocess.CalledProcessError:
    print("\n⚠️ Falló el push por autenticación (muy común si no tienes token guardado).")
    print("Solución rápida (HTTPS):")
    print("1) Crea un Personal Access Token (PAT) en GitHub (Settings > Developer settings > Personal access tokens).")
    print("2) Reintenta el push.")
    print("   Usuario: tu usuario de GitHub (jvasquez98)")
    print("   Password: pega el PAT (no tu contraseña normal).")
    raise

> git remote set-url origin https://github.com/jvasquez98/DocoBr.git
> git checkout main
> git reset
> git add index.html GenerateUI.ipynb images
> git commit -m chore: update carta UI + pdf download (2026-01-22 13:03)
> git push -u origin main
✅ Push completado.
